In [1]:
import pandas as pd
import numpy as np
import requests 
import matplotlib.pyplot as plt
import datetime 
from fbprophet import Prophet 
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from fbprophet.plot import plot_cross_validation_metric

from fbprophet.plot import plot_plotly
import plotly.offline as py
import streamlit as st

In [2]:
# Import data 
df = pd.read_json("https://api-pc6dbtrtla-uc.a.run.app/API/timeseries/usa")
df = df.rename(columns={'Totals as of Date': 'Date'})
df['Date'] = pd.to_datetime(df['Date']).dt.date
df['NewCases'] = df['Cases'] - df['Cases'].shift(1)
df['NewDeaths'] = df['Deaths'] - df['Deaths'].shift(1)

df_cases = df.loc[df["Cases"]>=200_000]
df_deaths = df.loc[df["Date"]>=datetime.date(2020,4,8)]

df_cases_fb = df_cases[["Date", "NewCases"]].rename(columns={"Date": "ds", "NewCases": "y"})
df_deaths_fb = df_deaths[["Date", "NewDeaths"]].rename(columns={"Date": "ds", "NewDeaths": "y"})

df

,ISO3,Country,Date,Cases,Deaths,Recovered,NewCases,NewDeaths
0,USA,US,2020-01-22,1,0,0,NaN,NaN
1,USA,US,2020-01-23,1,0,0,0.0,0.0
2,USA,US,2020-01-24,2,0,0,1.0,0.0
3,USA,US,2020-01-25,2,0,0,0.0,0.0
4,USA,US,2020-01-26,5,0,0,3.0,0.0
...,...,...,...,...,...,...,...,...
114,USA,US,2020-05-15,1442824,87530,250747,25050.0,1632.0
115,USA,US,2020-05-16,1467820,88754,268376,24996.0,1224.0
116,USA,US,2020-05-17,1486757,89562,272265,18937.0,808.0
117,USA,US,2020-05-18,1508308,90347,283178,21551.0,785.0


In [3]:
## Ed and Austin's Code

# state_list = ["Alabama", "Alaska", "Arizona", "Arkansas", "California", "Colorado", "Connecticut", "Delaware", "District of Columbia","Florida", "Georgia", "Hawaii", "Idaho", "Illinois", "Indiana", "Iowa", "Kansas", "Kentucky", "Louisiana", "Maine", "Maryland", "Massachusetts", "Michigan", "Minnesota", "Mississippi", "Missouri", "Montana", "Nebraska", "Nevada", "New Hampshire", "New Jersey", "New Mexico", "New York", "North Carolina", "North Dakota", "Ohio", "Oklahoma", "Oregon", "Pennsylvania", "Rhode Island", "South Carolina", "South Dakota", "Tennessee", "Texas", "Utah", "Vermont", "Virginia", "Washington", "West Virginia", "Wisconsin", "Wyoming"]
# state_selector = st.selectbox("Select State", state_list)

# state_str = state_selector.replace(' ', "%20")
# state_data_df = pd.read_json(f"https://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/{state_str}/allcounties")
# st.write(state_data_df[["State", "Totals as of Date", "Cases", "Deaths"]])

# state_data_df.State.unique()

In [4]:
## Chris' Code 

Alabama = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Alabama/")
Alaska = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Alaska/")
Arizona = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Arizona/")
Arkansas = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Arkansas/")
California = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/California/")
Colorado = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Colorado/")
Connecticut = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Connecticut/")
DC = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/District%20of%20Columbia/")
Delaware = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Delaware/")
Florida = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Florida/")
Georgia = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Georgia/")
Hawaii = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Hawaii/")
Idaho = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Idaho/")
Illinois = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Illinois/")
Indiana = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Indiana/")
Iowa = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Iowa/")
Kansas = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Kansas/")
Kentucky = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Kentucky/")
Louisiana = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Louisiana/")
Maine = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Maine/")
Maryland = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Maryland/")
Massachusetts = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Massachusetts/")
Michigan = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Michigan/")
Minnesota = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Minnesota/")
Mississippi = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Mississippi/")
Missouri = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Missouri/")
Montana = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Montana/")
Nebraska = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Nebraska/")
Nevada = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Nevada/")
New_Hampshire = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/New%20Hampshire/")
New_Jersey = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/New%20Jersey/")
New_Mexico = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/New%20Mexico/")
New_York = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/New%20York/")
North_Carolina = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/North%20Carolina/")
North_Dakota = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/North%20Dakota/")
Ohio = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Ohio/")
Oklahoma = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Oklahoma/")
Oregon = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Oregon/")
Pennsylvania = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Pennsylvania/")
Rhode_Island = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Rhode%20Island/")
South_Carolina = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/South%20Carolina/")
South_Dakota = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/South%20Dakota/")
Tennessee = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Tennessee/")
Texas = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Texas/")
Utah = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Utah/")
Vermont = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Vermont/")
Virginia = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Virginia/")
Washington = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Washington/")
West_Virginia = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/West%20Virginia/")
Wisconsin = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Wisconsin/")
Wyoming = pd.read_json("http://api-pc6dbtrtla-uc.a.run.app/API/us/timeseries/totals/Wyoming/")

In [11]:
Texas = Texas.rename(columns={'Totals as of Date': 'Date'})
Texas['Date'] = pd.to_datetime(Texas['Date']).dt.date
Texas['NewCases'] = Texas['Cases'] - Texas['Cases'].shift(1)
Texas['NewDeaths'] = Texas['Deaths'] - Texas['Deaths'].shift(1)

Texas_cases = Texas.loc[Texas["Date"]>=datetime.date(2020,4,1)]
Texas_deaths = Texas.loc[Texas["Date"]>=datetime.date(2020,4,8)]

Texas_cases = Texas_cases[["Date", "NewCases"]].rename(columns={"Date": "ds", "NewCases": "y"})
Texas_deaths= Texas_deaths[["Date", "NewDeaths"]].rename(columns={"Date": "ds", "NewDeaths": "y"})

Texas_cases

,ds,y
70,2020-04-01,546.0
71,2020-04-02,714.0
72,2020-04-03,665.0
73,2020-04-04,833.0
74,2020-04-05,642.0
75,2020-04-06,834.0
76,2020-04-07,882.0
77,2020-04-08,852.0
78,2020-04-09,1431.0
79,2020-04-10,897.0


In [6]:
# Predict function 

def predictcases(days):

    prophet = Prophet()
    prophet.fit(df_cases_fb)

    future = prophet.make_future_dataframe(periods=days)
    forecast = prophet.predict(future)
    df_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    fig_forecast = prophet.plot(forecast)

    return df_forecast

In [7]:
def predictdeaths(days):

    prophet = Prophet()
    prophet.fit(df_deaths_fb)

    future = prophet.make_future_dataframe(periods=days)
    forecast = prophet.predict(future)
    df_forecast = forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    fig_forecast = prophet.plot(forecast)

    return df_forecast


In [8]:
# Cross validation function

def cross_validate(df):

    prophet = Prophet()
    prophet.fit(df)

    df_cv = cross_validation(prophet, initial='30 days', period='4 days', horizon='7 days')
    df_performance = performance_metrics(df_cv)
    fig_performance = plot_cross_validation_metric(df_cv, metric='mape')

    return plt.show()


In [9]:
predict(df_cases_fb, 30)

NameError: name 'predict' is not defined